In [4]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

nba_ = pd.read_csv("/Users/ncapofari/Documents/CUNY_SPS/Capstone/%s" 
                  % "nba_scraper/data_sets/4factor_model_data.csv")

train_set, test_set = train_test_split(nba_, 
                                       test_size=0.2, 
                                       random_state=23)

#train_set = train_set[train_set['Seed_Diff'] > 0]
#train_set = train_set[train_set['Seed_Diff'] > 1]
train_set = train_set[train_set['Seed_Diff'] == 1]

In [5]:
#models

#nba_train = train_set["Seed_Diff"].copy().values.reshape(-1, 1)
#nba_train = train_set["Pyt_Win_Pct_Diff"].copy().values.reshape(-1, 1)
nba_train = train_set.drop(["Seed_Diff","Pyt_Win_Pct_Diff",
                            "Won_Series", "id"], axis=1)

f = "four_fact_RFC_is_one"

nba_labels = train_set["Won_Series"].copy()
nba_labels = (nba_labels == 1)
nba_test = test_set.copy()

In [12]:
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif

pipeline = Pipeline([
    ('std_scaler', StandardScaler()),
    ('select', SelectKBest(f_classif, 5))
])

nba_prepared = pipeline.fit_transform(nba_train, nba_labels)
print(nba_prepared.shape)

support = pipeline.named_steps['select'].get_support()
k_features = [] 

for b, feature in zip(support, list(nba_train.columns.values)):
    if b: k_features.append(feature)

print(k_features)

(115, 5)
['eFG..Misc', 'TOV..Misc', 'FT.FGA.Misc', 'DRB..Misc', 'ORB..Misc_._Opponent']


In [16]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

rfc_clf = RandomForestClassifier(random_state=23)
param_grid = {'n_estimators': [5, 10, 20, 50, 100],}
rfc_clf = GridSearchCV(rfc_clf, param_grid, cv=5)
rfc_clf.fit(nba_prepared, nba_labels)

print("Best estimator found by grid search:")
print(rfc_clf.best_estimator_)

Best estimator found by grid search:
RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=20, n_jobs=1,
            oob_score=False, random_state=23, verbose=0, warm_start=False)


In [17]:
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix, precision_score
from sklearn.metrics import roc_curve, roc_auc_score

def scores(m, fs, tgt, fname):
    cross_val_score(m, fs, tgt, cv=5)
    y_train_pred = cross_val_predict(m, fs, tgt, cv=5)
    print(confusion_matrix(tgt, y_train_pred))
    print("precision score:", precision_score(tgt, y_train_pred))
    fpr, tpr, thresholds = roc_curve(tgt, y_train_pred)
    print("roc_auc:", roc_auc_score(tgt, y_train_pred))
    temp = pd.DataFrame({"fpr":fpr, "tpr":tpr, "thresholds":thresholds})
    temp.to_csv("scores/"+fname+".csv")
    preds = pd.DataFrame({"Prediction":y_train_pred})
    x = pd.concat([preds.reset_index(drop=True), 
                   train_set.reset_index(drop=True)], axis=1)
    x.to_csv("results/"+fname+".csv")

In [18]:
scores(rfc_clf, nba_prepared, nba_labels, f)

[[41 19]
 [29 26]]
precision score: 0.5777777777777777
roc_auc: 0.578030303030303
